# Searching for Be stars in binaries with compact objects
## Catalog cross-matching
### Caden Gobat, The George Washington University

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, astropy.units as u, pickle
from astropy.io import fits, ascii as ascii_io
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [2]:
def hms_to_dec(hms):
    h,m,s = pd.to_numeric(hms.split()) # hms will always be between 0 and +24 hours
    deg = ((s/60 + m)/60 + h)*15 # 360 degrees/24 hours = 15 deg/hr
    return float(deg)
    
def dms_to_dec(dms):
    if dms[0] == "-": # since negative degrees are possible
        sign = -1
    else:
        sign = 1
    d,m,s = pd.to_numeric(dms.split())    
    dmag = np.abs(d) # use magnitudes only and reapply sign after
    deg = sign*((s/60 + m)/60 + dmag)
    return float(deg)

### Initialize ViZier query instance

In [3]:
v = Vizier(columns=['all', '_RAJ2000', '_DEJ2000', "_r"])
v.ROW_LIMIT = -1 # no row limit

### Load Be star catalog

In [4]:
#Jaschek1982 = v.query_constraints(catalog="III/67A/catalog")[0] # old Be catalog; Table object
BeSS = ascii_io.read("./catalogs/BeSS_catalog.csv", format="csv", header_start=0, data_start=1)
BeSS["_RAJ2000"] = [hms_to_dec(hms) for hms in BeSS["RA"]]*u.degree
BeSS["_DEJ2000"] = [dms_to_dec(dms) for dms in BeSS["DEC"]]*u.degree
BeSS

#,Be star,Category,RA,DEC,V,Type,vsini,Nb of spectra in BeSS,_RAJ2000,_DEJ2000
,,,,,,,,,deg,deg
int32,str26,str19,str11,str12,float64,str10,int32,int32,float64,float64
1,BD+62 2346,Classical,00 01 24.70,63 30 15.73,9.73,B0Ve,--,3,0.35291666666666666,63.50436944444444
2,HD 224905,Classical,00 01 38.63,60 26 59.72,8.47,B1Vne,--,85,0.4109583333333333,60.44992222222222
3,HD 225095,Classical,00 03 27.15,55 33 03.23,7.95,B2IVne,--,234,0.863125,55.550897222222225
4,2 Cet,Classical,00 03 44.39,-17 20 09.57,4.543,B9IVne,--,95,0.9349583333333333,-17.335991666666665
5,10 Cas,Classical,00 06 26.54,64 11 46.21,5.567,B9IIIe,125,292,1.6105833333333333,64.19616944444445
6,BD+59 2829,Classical,00 06 48.30,60 36 00.83,9.86,B1Ve,--,2,1.70125,60.600230555555555
7,BD+62 1,Classical,00 07 33.15,63 04 49.18,10.46,B2IVpe,--,2,1.8881250000000003,63.080327777777775
8,BD+62 11,Classical,00 10 46.92,63 10 22.57,9.6,B5Ve,--,6,2.6955,63.17293611111111


### Crossmatch with Gaia data

In [5]:
Gaia_BeSS_1as = v.query_region(BeSS, catalog="I/345/gaia2", radius="1s")
Gaia_BeSS_1as[0]

_q,_RAJ2000,_DEJ2000,_r,DR2Name,RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,SolID,Source,RandomI,Epoch,Plx,e_Plx,RPlx,pmRA,e_pmRA,pmDE,e_pmDE,RADEcor,RAPlxcor,RApmRAcor,RApmDEcor,DEPlxcor,DEpmRAcor,DEpmDEcor,PlxpmRAcor,PlxpmDEcor,pmRApmDEcor,NAL,NAC,NgAL,NbAL,gofAL,chi2AL,epsi,sepsi,Solved,APF,WAL,pscol,e_pscol,fvarpi,MatchObsA,Nper,amax,type,MatchObs,Dup,o_Gmag,FG,e_FG,RFG,Gmag,e_Gmag,o_BPmag,FBP,e_FBP,RFBP,BPmag,e_BPmag,o_RPmag,FRP,e_FRP,RFRP,RPmag,e_RPmag,E_BR_RP_,Mode,BP-RP,BP-G,G-RP,RV,e_RV,o_RV,Tefftemp,loggtemp,__Fe_H_temp,Var,GLON,GLAT,ELON,ELAT,fPriam,Teff,b_Teff,B_Teff,AG,b_AG,B_AG,E_BP-RP_,b_E_BP-RP_,B_E_BP-RP_,fFLAME,Rad,b_Rad,B_Rad,Lum,b_Lum,B_Lum,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000
,deg,deg,arcsec,,deg,mas,deg,mas,,,,yr,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,e-/s,e-/s,,mag,mag,,e-/s,e-/s,,mag,mag,,e-/s,e-/s,,mag,mag,,,mag,mag,mag,km / s,km / s,,K,[cm/s2],,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,Rsun,Rsun,Rsun,Lsun,Lsun,Lsun,deg,mas,deg,mas
int32,float64,float64,float64,bytes28,float64,float64,float64,float64,int64,int64,int64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int16,int16,int16,int16,float64,float64,float32,float32,int16,uint8,float64,float64,float32,float32,int16,int16,float64,uint8,int16,uint8,int16,float32,float32,float64,float64,float64,int16,float32,float32,float64,float64,float64,int16,float32,float32,float64,float64,float64,float32,uint8,float64,float64,float64,float64,float32,int16,float32,float32,float32,bytes13,float64,float64,float64,float64,int32,float64,float64,float64,float32,float32,float32,float32,float32,float32,int32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64
1,0.3528934968748,63.5043646703320,0.080,Gaia DR2 431593257757382912,0.35290285715,0.0366,63.50434817961,0.0389,1635721458409799680,431593257757382912,544894291,2015.5,0.8406,0.0470,17.8938,0.970,0.072,-3.830,0.070,-0.1428,0.2064,-0.4061,0.1264,-0.0229,0.0835,-0.3013,-0.2580,0.2216,-0.2201,298,298,280,18,26.2873,1464.89,0.186,14,31,1,22.8036,1.6097,0.0106,0.070,34,19,0.050373,0,38,0,333,3.55e+06,2580,1376.000,9.3128,0.0008,36,1.942e+06,6102,318.213,9.6309,0.0034,36,2.335e+06,5745,406.477,8.8411,0.0027,1.205,0,0.7898,0.3181,0.4717,--,--,0,--,--,--,NOT_AVAILABLE,117.38675266737,1.17248191362,38.78659064532,55.08722418148,100001,5880.50,5818.00,5961.00,--,--,--,--,--,--,200111,13.53,13.17,13.82,197.252,181.853,212.651,0.35289349687,1.1216,63.50436467030,1.0800
2,0.4109778695160,60.4499236685292,0.022,Gaia DR2 429341153012514176,0.41095008342,0.0260,60.44991749452,0.0290,1635721458409799680,429341153012514176,136432035,2015.5,0.6544,0.0366,17.8987,-3.183,0.055,-1.434,0.055,-0.1581,0.1448,-0.3160,0.1465,0.0826,0.1259,-0.3798,-0.2228,0.0546,-0.2339,292,292,284,8,11.7107,655.95,0.000,0,31,1,159.3133,1.7528,0.0071,0.084,33,18,0.041675,0,39,0,332,8.226e+06,4060,2025.919,8.4004,0.0005,37,5.534e+06,7525,735.390,8.4938,0.0015,38,4.056e+06,5415,749.030,8.2417,0.0014,1.166,0,0.2521,0.0934,0.1588,--,--,0,--,--,--,NOT_AVAILABLE,116.82603118642,-1.83017728659,35.30748935611,52.81929468584,100001,8364.00,8137.00,8688.50,0.6860,0.2140,1.0571,0.3213,0.1132,0.5570,--,--,--,--,--,--,--,0.41097786952,0.8457,60.44992366850,0.8594
3,0.8631149750008,55.5508984387620,0.057,Gaia DR2 420875841188703360,0.86309871119,0.0271,55.55089190034,0.0264,1635721458409799680,420875841188703360,875116711,2015.5,0.7693,0.0398,19.3084,-2.137,0.052,-1.519,0.047,-0.2711,0.3124,-0.3908,0.2143,-0.0974,0.1362,-0.3554,-0.2160,0.3433,-0.2285,393,393,386,7,20.0169,1242.25,0.066,1.7,31,0,46.8856,1.8032,0.0066,-0.030,44,18,0.038378,0,47,1,417,1.313e+07,1.067e+04,1230.775,7.8924,0.0009,46,9.648e+06,2.05e+04,470.703,7.8903,0.0023,46,5.581e+06,1.792e+04,311.384,7.8951,0.0035,1.160,0,-0.0047,-0.0021,-0.0027,--,--,0,--,--,--,NOT_AVAILABLE,116.13327877423,-6.68616184481,30.70013832736,48.8677

### XMM, Chandra, and Swift/XRT catalogs
Load the full catalogs from locally stored copies. Crossmatch from vizier in the cloud.

In [6]:
with open("./catalogs/xmm4dr9s.pkl","rb") as file:
    XMM = pickle.load(file)
with open("./catalogs/csc2master.pkl","rb") as file:
    CSC = pickle.load(file)
with open("./catalogs/2sxpscle.pkl","rb") as file:
    XRT = pickle.load(file)

cat_ids = ["IX/59/xmm4dr9s","IX/57/csc2master","IX/58/2sxpscle"]
xmatch = [None, None, None]
for i in range(len(cat_ids)):
    xmatch[i] = v.query_region(BeSS, catalog=cat_ids[i], radius="1s")
    print("Successfully matched BeSS with",cat_ids[i])
[XMM_xmatch, CSC_xmatch, XRT_xmatch] = xmatch

Successfully matched BeSS with IX/59/xmm4dr9s
Successfully matched BeSS with IX/57/csc2master
Successfully matched BeSS with IX/58/2sxpscle


In [7]:
print("BeSS entries coincident with XMM sources:\n",XMM_xmatch[0]["_q","_4XMM","_RAJ2000","_DEJ2000","_r"],"\n")
print("BeSS entries coincident with Chandra sources:\n",CSC_xmatch[0]["_q","_2CXO","_RAJ2000","_DEJ2000","_r"],"\n")
print("BeSS entries coincident with Swift/XRT sources:\n",XRT_xmatch[0]["_q","IAUName","_RAJ2000","_DEJ2000","_r"],"\n")

BeSS entries coincident with XMM sources:
  _q       _4XMM         _RAJ2000     _DEJ2000     _r  
                          deg          deg      arcsec
---- ---------------- ------------ ------------ ------
  60 J005044.7-731605  12.68631111 -73.26818194  0.314
  66 J005152.0-731033  12.96667611 -73.17608111  0.076
  69 J005205.6-722603  13.02369000 -72.43431194  0.507
  82 J005323.8-722715  13.34927194 -72.45433194  0.583
 117 J005446.3-722523  13.69321694 -72.42318194  0.728
 217 J005642.4+604300  14.17688194  60.71668806  0.606
 258 J010120.6-721119  15.33617194 -72.18862000  0.595
 287 J011108.4-731646  17.78537500 -73.27950000  0.530
 382 J014700.2+612123  26.75083500  61.35655806  0.233
 465 J020845.4+650214  32.18933111  65.03748500  0.216
 ...              ...          ...          ...    ...
1891 J180411.1-242145 271.04633000 -24.36255000  0.698
1893 J180425.7-242309 271.10750000 -24.38587389  0.966
1896 J180432.9-241844 271.13716306 -24.31233194  0.230
1908 J180827.0-195207 

### Manual crossmatching using `astropy.coordinates.SkyCoord`

In [8]:
BeSS_coords = SkyCoord(np.array(BeSS["_RAJ2000","_DEJ2000"]), unit=u.degree, frame='icrs') 
XMM_coords = SkyCoord(np.array(XMM["_RAJ2000","_DEJ2000"]), unit=u.degree, frame='icrs')
CSC_coords = SkyCoord(np.array(CSC["_RAJ2000","_DEJ2000"]), unit=u.degree, frame='icrs')
XRT_coords = SkyCoord(np.array(XRT["_RAJ2000","_DEJ2000"]), unit=u.degree, frame='icrs')

In [9]:
XMM_matches = XMM_coords.match_to_catalog_sky(BeSS_coords)
CSC_matches = CSC_coords.match_to_catalog_sky(BeSS_coords)
XRT_matches = XRT_coords.match_to_catalog_sky(BeSS_coords)

In [10]:
XMM_filt = XMM_matches[1] < 1*u.arcsecond
CSC_filt = CSC_matches[1] < 1*u.arcsecond
XRT_filt = XRT_matches[1] < 1*u.arcsecond

In [11]:
XMM["_q"] = XMM_matches[0]+1 # need to match the 1-indexed label used astroquery and BeSS
XMM["_r"] = XMM_matches[1]
CSC["_q"] = CSC_matches[0]+1
CSC["_r"] = CSC_matches[1]
XRT["_q"] = XRT_matches[0]+1
XRT["_r"] = XRT_matches[1]

XMM_matched = XMM[XMM_filt]
CSC_matched = CSC[CSC_filt]
XRT_matched = XRT[XRT_filt]

#### Compare to TOPCAT's Swift crossmatch

In [12]:
topcat_fits = fits.open("BeSS_Swift2xPC_crossmatch.fits")
topcat_xmatch = pd.DataFrame(topcat_fits[1].data)
topcat_xmatch

,col1,col2,col3,col4,col5,col6,col7,col8,col9,RA_1,...,MatchInSwiftFT,MatchIn1SWXRT,MatchInXRTGRB,MatchInSDSSQSO,MatchIn2MASS,MatchInUSNOB1,MatchIn2CSC,MatchIn1SXPS,MatchInALLWISE,Separation
0,60,DZ Tuc,Classical,00 50 44.69,-73 16 05.16,15.440000,Be,-32768,0,12.686208,...,0,1,0,0,1,1,1,1,1,0.711177
1,71,2E 0050.4-7335,Classical,00 52 13.99,-73 19 18.84,14.670000,Be,-32768,0,13.058292,...,0,1,0,0,1,1,1,1,1,0.842063
2,82,CXOU J005323.8-722715,Classical,00 53 23.90,-72 27 15.12,16.190001,Be,-32768,0,13.349583,...,0,1,0,0,1,1,1,1,0,0.999087
3,92,Cl* NGC 330 ELS 065055,Classical,00 53 55.34,-72 26 45.30,14.720000,B1IIIe,420,0,13.480583,...,0,1,0,0,1,1,1,1,0,0.702086
4,258,RX J0101.3-7211,Classical,01 01 20.81,-72 11 18.96,15.490000,Be,-32768,0,15.336708,...,0,0,0,0,1,1,0,1,1,0.552364
5,597,V615 Cas,Classical,02 40 31.67,61 13 45.61,10.800000,B0Ve,-32768,5,40.131958,...,0,1,0,0,1,1,1,1,1,0.631448
6,705,Menkhib,Classical,03 58 57.90,35 47 27.71,4.060000,O7.5IIIe,213,4655,59.741250,...,0,0,0,0,1,1,0,1,1,0.911434
7,758,RX J0516.0-6916,Classical,05 16 00.10,-69 16 09.12,15.000000,Be,-32768,0,79.000417,...,0,0,0,0,1,0,0,0,0,0.549697
8,842,RX J0531.2-6609,Classical,05 31 13.30,-66 07 05.16,14.400000,B0.7Ve,-32768,0,82.805417,...,0,1,0,0,1,1,0,1,1,0.531911
9,892,V725 Tau,Classical,05 38 54.57,26 18 56.83,9.190000,O9.7IIe,-32768,11,84.727375,...,0,1,0,0,1,1,1,1,1,0.714977
